Расчет расстояний между городами по их координатам 

In [3]:
import pandas as pd
import numpy as np
from time import time
t1 = time()

In [4]:
df = pd.read_csv('cities.csv')

name - название города, lat - координата широты, lng  - координата долготы.

результат: название 1-го города, название 2-го города, расстояние между ними. (без дубликатов)

In [5]:
df

,name,lat,lng
0,New York,40.6943,-73.9249
1,Los Angeles,34.1141,-118.4068
2,Chicago,41.8375,-87.6866
3,Miami,25.7840,-80.2101
4,Dallas,32.7935,-96.7667
...,...,...,...
30839,Ironville,40.6586,-78.2155
30840,Newkirk,35.0635,-104.2715
30841,Falcon Village,26.5652,-99.1341
30842,Hidden Lakes,40.5479,-82.7632


In [13]:
# срез городов по количеству, берем 1500 городов
slice_of_cities = df.head(1500)

In [12]:
# делаем все возможные комбинации городов (чтобы из каждого города найти расстояние во все остальные города)
df_cities = slice_of_cities.merge(slice_of_cities, how='cross')
# отбираем строки, без повторяющихся городов (чтобы мы не искали расстояние из города в этот же город)
df_cities = df_cities[df_cities['name_x'] != df_cities['name_y']]

In [8]:
# радиус земли в км
R = 6371.0
# находим разницу между ширинами и долготами городов
df_cities['dlat'] = df_cities['lat_y'] - df_cities['lat_x']
df_cities['dlng'] = df_cities['lng_y'] - df_cities['lng_x']

In [9]:
# находим расстояния по формуле Гаверсинусов (Haversine formula)
df_cities['inside_f'] = np.sin(df_cities['dlat'] / 2)**2 + np.cos(df_cities['lat_x']) * np.cos(df_cities['lat_y']) * np.sin(df_cities['dlng'] / 2)**2 
df_cities['haversine_f'] = 2 * np.arcsin(np.sqrt(df_cities['inside_f'])) 
df_cities['distance'] = df_cities['haversine_f'] * R

In [10]:
# другой пример рассчета
# df_cities['antipode'] = np.cos(df_cities['lat_x']) * np.cos(df_cities['lat_y']) * np.sin(df_cities['dlng'] / 2)**2 + np.sin(df_cities['dlat'] / 2)**2
# df_cities['distance'] = np.arctan2(df_cities['antipode']**0.5, (1-df_cities['antipode'])**0.5) * 2 * R

In [11]:
df_final = df_cities[['name_x', 'name_y', 'distance']]
df_final

,name_x,name_y,distance
1,New York,Los Angeles,3570.949742
2,New York,Chicago,9532.540782
3,New York,Miami,14933.159902
4,New York,Dallas,8276.008274
5,New York,Houston,11265.319568
...,...,...,...
2249994,Southlake,Garner,17439.808355
2249995,Southlake,Gurnee,19553.104283
2249996,Southlake,Opelika,1935.184400
2249997,Southlake,Parkville,714.460009


In [11]:
time() - t1

0.8441863059997559